<img src="https://datasciencedegree.wisconsin.edu/wp-content/themes/data-gulp/images/logo.svg" width="300">


# Capstone Project -- NodeXL Data Collection
## Matt Peterson - DS 785
### 05/06/22

---

## Setup for file system

#### Gain access to required libraries

In [1]:
import re
import os
from pathlib import Path
import statistics
import json
import xlrd
import pandas as pd

#### Set input and output file names

In [2]:
json_output_filename = 'influential_vr_accounts.json'
avg_top_items_output_filename = 'users_with_average_top_items.csv'
avg_top_items_column_names = ("USERNAME", 
                              "NAME",
                              "AVG_IN_DEGREE", 
                              "NUM_DAYS_WITH_TOP_N_IN_DEGREE",
#                               "AVG_OUT_DEGREE", 
#                               "NUM_DAYS_WITH_TOP_N_OUT_DEGREE",
                              "AVG_BETWEENNESS", 
                              "NUM_DAYS_WITH_TOP_N_BETWEENNESS",
#                               "AVG_CLOSENESS", 
#                               "NUM_DAYS_WITH_TOP_N_CLOSENESS",
#                               "AVG_EIGENVECTOR", 
#                               "NUM_DAYS_WITH_TOP_N_EIGENVECTOR",
                              "AVG_PAGE_RANK", 
                              "NUM_DAYS_WITH_TOP_N_PAGE_RANK",
                              "DESCRIPTION");

#### Set input and output directory paths

In [3]:
file_dir = cwd = Path.cwd()
node_xl_folder = os.path.join(file_dir, 'input')
avg_top_items_output_file = os.path.join(file_dir, 'output', avg_top_items_output_filename)
json_output_file = os.path.join(file_dir, 'output', json_output_filename)

---

## Use ```json``` to Read Influential User Dictionary From File

#### Function that returns a user dictionary retrieved from JSON file OR an empty dict if none exists

In [4]:
def get_user_dictionary():
    try:
        with open(json_output_file) as json_file:
            user_dict = json.load(json_file)
    except FileNotFoundError:
        user_dict = {}
    return user_dict

In [5]:
user_dict = get_user_dictionary()

---

## Use ```pandas``` to Read NodeXL Info from File

#### Function that parses the Vertices sheet in NodeXL file, creates a dict, then adds it to the list

In [6]:
def update_user_dict(user_dict, df, measure, file_date):
    
    for row in df.itertuples():
        username = row[1]
        in_degree = row[2]
#         out_degree = row[3]
        betweenness = row[3]
#         closeness = row[5]
#         eigenvector = row[6]
        pagerank = row[4]
        name = row[5]
        description = row[6]
        
        #update user dictionary for measure value tracking
        if username in user_dict:
            if not file_date in user_dict[username]['records']:
                user_dict[username]['records'][file_date] = {
                    'in-degree': in_degree,
#                     'out-degree': out_degree,
                    'betweenness': betweenness, 
#                     'closeness': closeness,
#                     'eigenvector': eigenvector,
                    'pagerank': pagerank
                }
        else:
            user_dict[username] = {
                'name': name, 
                'description': description, 
                'records': {file_date:
                {
                    'in-degree': in_degree,
#                     'out-degree': out_degree,
                    'betweenness': betweenness, 
#                     'closeness': closeness,
#                     'eigenvector': eigenvector,
                    'pagerank': pagerank
                 }},
                'counts': {
                    'in-degree': 0,
#                     'out-degree': 0,
                    'betweenness': 0, 
#                     'closeness': 0,
#                     'eigenvector': 0,
                    'pagerank': 0
                }}
            
        user_dict[username]['counts'][measure] += 1

In [7]:
def add_top_n_users_to_dict_list_by_measure(user_dict, df, measure, top_n, file_date):
    sorted_df = df.sort_values(by=measure, ascending=False)
    top_n_records = sorted_df.iloc[:top_n]
    update_user_dict(user_dict, top_n_records, measure, file_date)

#### Function that:
#### 1. Builds a dict list containing each user record having the top N measures in the vertices sheet from each NodeXL file in the directory
#### 2. Updates the user dictionary with new records

In [8]:
def format_text(text):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|(\w+:\/\/\S+)"," ",str(text)).split())

In [9]:
def update_user_dict_with_top_n_items(user_dict, node_xl_folder, top_n):
    
    for node_xl_file in os.listdir(node_xl_folder):

        #get date from filename
        file_date = node_xl_file[3:13]

        #specify filepath
        input_file = os.path.join(file_dir, 'NodeXL Sheets', 'Unprocessed', node_xl_file)

        #open NodeXL workbook and store the Vertices sheet as a pandas dataframe
        df = pd.read_excel(input_file, sheet_name='Vertices', header=1,
                           usecols=['Vertex',
                                    'In-Degree',
#                                     'Out-Degree',
                                    'Betweenness Centrality',
#                                     'Closeness Centrality',
#                                     'Eigenvector Centrality',
                                    'PageRank',
                                    'Name',
                                    'Description'])
        df.columns = ['vertex', 
                      'in-degree', 
#                       'out-degree', 
                      'betweenness',
#                       'closeness',
#                       'eigenvector', 
                      'pagerank',
                      'name',
                      'description']
        
        #clean up format in description so it can be written to JSON
        df['description'] = df['description'].apply(format_text)
        df['name'] = df['name'].apply(format_text)
                
        add_top_n_users_to_dict_list_by_measure(user_dict, df, 'in-degree', top_n, file_date)
#         add_top_n_users_to_dict_list_by_measure(user_dict, df, 'out-degree', top_n, file_date)
        add_top_n_users_to_dict_list_by_measure(user_dict, df, 'betweenness', top_n, file_date)
#         add_top_n_users_to_dict_list_by_measure(user_dict, df, 'closeness', top_n, file_date)
#         add_top_n_users_to_dict_list_by_measure(user_dict, df, 'eigenvector', top_n, file_date)
        add_top_n_users_to_dict_list_by_measure(user_dict, df, 'pagerank', top_n, file_date)

#### Function that builds a dict list containing each user and their average top item measures

In [10]:
def get_measure_average(user_dict, user, measure):
    return statistics.mean([records[measure] for date, records in user_dict[user]['records'].items()])

In [11]:
def get_avg_top_items_dict_list(user_dict, column_names):
    dict_list = []
    
    for user in user_dict:

        dict_list.append(dict(zip(column_names, [user, user_dict[user]['name'],
            get_measure_average(user_dict, user, 'in-degree'), user_dict[user]['counts']['in-degree'],
#             get_measure_average(user_dict, user, 'out-degree'), user_dict[user]['counts']['out-degree'],
            get_measure_average(user_dict, user, 'betweenness'), user_dict[user]['counts']['betweenness'],
#             get_measure_average(user_dict, user, 'closeness'), user_dict[user]['counts']['closeness'],
#             get_measure_average(user_dict, user, 'eigenvector'), user_dict[user]['counts']['eigenvector'],
            get_measure_average(user_dict, user, 'pagerank'), user_dict[user]['counts']['pagerank'],
            user_dict[user]['description']])))

    return dict_list

In [12]:
top_n_items = 200

#get the dict list containing an instance of each top item measure and update the user dictionary with new instances
update_user_dict_with_top_n_items(user_dict, node_xl_folder, top_n_items)

#convert the user dictionary into a dict list containing each user and their average top item measures
avg_top_items_dict_list = get_avg_top_items_dict_list(user_dict, avg_top_items_column_names)

In [13]:
avg_top_items_dict_list

[{'USERNAME': 'metawarofficial',
  'NAME': 'Metawar',
  'AVG_IN_DEGREE': 113.83333333333333,
  'NUM_DAYS_WITH_TOP_N_IN_DEGREE': 11,
  'AVG_BETWEENNESS': 535773.5,
  'NUM_DAYS_WITH_TOP_N_BETWEENNESS': 2,
  'AVG_PAGE_RANK': 52.45332366666667,
  'NUM_DAYS_WITH_TOP_N_PAGE_RANK': 12,
  'DESCRIPTION': 'Telegram Links: Website:'},
 {'USERNAME': 'kichchakiran',
  'NAME': 'KICHCHA KIRAN',
  'AVG_IN_DEGREE': 93.45454545454545,
  'NUM_DAYS_WITH_TOP_N_IN_DEGREE': 11,
  'AVG_BETWEENNESS': 236450.04342536363,
  'NUM_DAYS_WITH_TOP_N_BETWEENNESS': 3,
  'AVG_PAGE_RANK': 12.757469545454546,
  'NUM_DAYS_WITH_TOP_N_PAGE_RANK': 8,
  'DESCRIPTION': 'SHIRDI SAI BABA||KICHCHA BOSS||PRIYA AKKA||SUDHA MURTHY||DEEPIKA PADUKONE||RANVEER SINGH|| 🇮🇳 INDAIN||ಕನ್ನಡಿಗ💛❤️|'},
 {'USERNAME': 'kicchasudeep',
  'NAME': 'Kichcha Sudeepa',
  'AVG_IN_DEGREE': 154.67857142857142,
  'NUM_DAYS_WITH_TOP_N_IN_DEGREE': 28,
  'AVG_BETWEENNESS': 1071633.41022575,
  'NUM_DAYS_WITH_TOP_N_BETWEENNESS': 11,
  'AVG_PAGE_RANK': 19.51252492

---

## Use ```pandas``` for Data Frame Creation

In [14]:
#create data frame for average top items by user
avg_top_items_df = pd.DataFrame(avg_top_items_dict_list)

---

## Export data frames to ```.csv``` file for analysis

In [15]:
# save df to a .csv file 
avg_top_items_df.to_csv(avg_top_items_output_file, index=False)

---

## Export influential VR account dictionary to ```.json``` file for record keeping

In [16]:
with open(json_output_file, "w") as outfile:
    json.dump(user_dict, outfile)